TRIGGERS

In [ ]:
-- TRIGGER PARA
CREATE TRIGGER tr_Bichos on Bichos FOR INSERT as
begin
DECLARE @tipo VARCHAR(20), @especie VARCHAR(20)
    SELECT @tipo = bicho_tipo from inserted 
    SELECT @especie = bicho_especie from inserted
    if()

    BEGIN
    ROLLBACK tran
    RAISERROR('No es compatible el tipo con la especie del bicho agregado')
    end
end

CREATE TRIGGER tr_

In [ ]:
USE bichos_de_bolsillo
GO

-- TRIGGER PARA ACTUALIZAR EL GANADOR DEL COMBATE DESPUÉS DE CADA RONDA
CREATE TRIGGER dbo.SetGanador_T ON dbo.Rondas 
AFTER INSERT 
AS 
BEGIN
    DECLARE @ganador int, @combate_id int, @usuario_uno int, @usuario_dos int

    --get combate_id from inserted
    SET @combate_id = (SELECT combate_id
    FROM inserted)

    -- get users
    SET @usuario_uno = (SELECT usuario_uno
    FROM Combates
    WHERE combate_id = @combate_id)

    SET @usuario_dos = (SELECT usuario_dos
    FROM Combates
    WHERE combate_id = @combate_id)

    -- get ganador
    SET @ganador = dbo.GetGanadorCombate(@combate_id, @usuario_uno, @usuario_dos)

    -- Set ganador in the table
    UPDATE Combates 
    SET ganador = @ganador
    WHERE combate_id = (SELECT combate_id
    FROM INSERTED)
END

GO

-- TRIGGER PARA AUMENTAR DE NIVEL AL GANADOR DE UNA RONDA
CREATE TRIGGER dbo.AumentarNivel_T ON Rondas 
AFTER INSERT
AS 
BEGIN
    DECLARE @ganador int, @bicho1 int, @bicho2 int, @daño1 int, @daño2 int;

    SET @ganador = (SELECT ganador
    FROM inserted)

    SET @bicho1 = (SELECT bicho_uno
    FROM inserted)

    SET @bicho2 = (SELECT bicho_dos
    FROM inserted)

    SET @daño1 = (SELECT nivel_daño_uno
    FROM inserted)

    SET @daño2 = (SELECT nivel_daño_dos
    FROM inserted)

    UPDATE Bichos SET bicho_salud = ROUND(((bicho_salud - @daño2) / 2), 0) WHERE bicho_id = @bicho1
    UPDATE Bichos SET bicho_salud = ROUND((bicho_salud - @daño1), 0) WHERE bicho_id = @bicho2

    IF(@daño1>@daño2)
        UPDATE Bichos SET bicho_nivel = (bicho_nivel + 1) WHERE bicho_id = @bicho1
    ELSE
        UPDATE Bichos SET bicho_nivel = (bicho_nivel + 1) WHERE bicho_id = @bicho2
END

GO

-- TRIGGER PARA ACTUALIZAR VALORES DE PROPIETARIOS CUANDO SE INSERTA EN INTERCAMBIOS
CREATE TRIGGER dbo.ActualizarPropietario_T ON dbo.Intercambios 
AFTER INSERT 
AS
BEGIN
    UPDATE Bichos SET bicho_propietario = (SELECT usuario_dos
    FROM inserted) WHERE bicho_id = (SELECT bicho_uno
    FROM inserted)

    UPDATE Bichos SET bicho_propietario = (SELECT usuario_uno
    FROM inserted) WHERE bicho_id = (SELECT bicho_dos
    FROM inserted)
END 

GO

-- TRIGGER PARA ACTUALIZAR VALORES DE PROPIETARIOS CUANDO SE INSERTA EN INTERCAMBIOS
CREATE TRIGGER dbo.RecolectaBicho_T ON dbo.Recolecciones 
AFTER INSERT 
AS
BEGIN
    UPDATE Bichos SET bicho_propietario = (SELECT recoleccion_usuario
    FROM inserted) WHERE bicho_id = (SELECT recoleccion_bicho
    FROM inserted)
END 

GO
